In [1]:
import pandas as pd
import numpy as np
import ipywidgets as ipw
import plotly.graph_objects as go

import tardis
from tardis.visualization import CustomAbundanceWidget
from tardis.model.density import calculate_power_law_density, calculate_exponential_density
from tardis.util.base import (
    atomic_number2element_symbol,
    quantity_linspace
)

/Users/yuyizheng/opt/anaconda3/envs/tardis/lib/python3.7/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
# sim = tardis.run_tardis('my_yml.yml')

### Load from Simulation object

In [3]:
# CustomAbundanceWidget.from_simulation(sim)

### If config file contains 'csvy_model' module, 'from_yml()' fails

In [4]:
# CustomAbundanceWidget.from_yml('my_yml.yml')

### Call 'from_csvy()' then

In [5]:
# my_cawidget = CustomAbundanceWidget.from_csvy('my_csvy.csvy')

### How abundance data is stored in this class

In [6]:
# my_cawidget.abundance

### Load from YAML file

In [7]:
cawidget = CustomAbundanceWidget.from_yml('tardis_example.yml')

In [8]:
cawidget.density

<Quantity [1.97052717e+03, 1.33603184e+01, 1.01466582e+01, 7.78662077e+00,
           6.03344417e+00, 4.71712224e+00, 3.71894570e+00, 2.95498192e+00,
           2.36519135e+00, 1.90615556e+00, 1.54615391e+00, 1.26178882e+00,
           1.03564575e+00, 8.54652716e-01, 7.08918432e-01, 5.90901467e-01,
           4.94810562e-01, 4.16168092e-01, 3.51489643e-01, 2.98046988e-01,
           2.53691438e-01] g / cm3>

In [9]:
cawidget.velocity

<Quantity [11000., 11450., 11900., 12350., 12800., 13250., 13700., 14150.,
           14600., 15050., 15500., 15950., 16400., 16850., 17300., 17750.,
           18200., 18650., 19100., 19550., 20000.] km / s>

In [10]:
cawidget.abundance

,,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
atomic_number,mass_number,,,,,,,,,,,,,,,,,,,,
8,,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19
12,,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03
14,,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52
16,,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19,0.19
18,,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04,0.04
20,,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03,0.03


### Codes to generate GUI

In [11]:
abundance = cawidget.abundance.copy()
velocity = cawidget.velocity.copy()
density = cawidget.density.copy()
d_type = cawidget.density

no_of_elements = abundance.shape[0]
no_of_shells = abundance.shape[1]
# shell_no = 1

def get_symbols(abundance):
    str_symbols = np.array(abundance.index.get_level_values(0).map(atomic_number2element_symbol))
    str_mass = np.array(abundance.index.get_level_values(1), dtype='str')
    return(np.add(str_symbols, str_mass))

elements = get_symbols(abundance)

In [12]:
from IPython.display import display

fig = go.FigureWidget()

l = list(range(1, no_of_shells+1))
dropdown = ipw.Dropdown(options=l, 
                        description='Shell No. ', 
                        layout=ipw.Layout(width='160px')
                       )

# shell_no = dropdown.value

btn_previous = ipw.Button(icon="chevron-left", 
                          disabled=True, 
                          layout=ipw.Layout(width='30px', height='30px')
                         )
btn_next = ipw.Button(icon="chevron-right", 
                      layout=ipw.Layout(width='30px', height='30px')
                     )

locks = [ipw.Checkbox(indent=False, 
                      layout=ipw.Layout(width='30px')
                     ) for element in elements]

items = [ipw.BoundedFloatText(min=0, 
                              max=1, 
                              step=0.01, 
                              description=element) 
         for element in elements]

btn_normalize = ipw.Button(description='Normalize',
                           icon='cog', 
                           layout = ipw.Layout(width='100px',
                                               margin='0 0 0 50px')
                          )

abundance_edit_container = ipw.HBox([ipw.VBox(items), ipw.VBox(locks)])

### Plot

In [13]:
# generate abundance vs velocity plot using Plotly
plot_output = ipw.Output()

@plot_output.capture(clear_output=True)
def generate_abundance_density_plot():
    title = "Abundance/Density vs Velocity"
    data = abundance.T
    
    fig.add_trace(
        go.Scatter(
            x=velocity[1:],
            y=density[1:],
            mode="lines+markers",
            name="<b>Density</b>",
            yaxis="y2",
            line=dict(color="black"),
            marker_symbol="square",
        ),
    )
    
    for i in range(no_of_elements):
        fig.add_trace(
            go.Scatter(
                x=velocity[1:],
                y=data.iloc[:,i],
                mode="lines+markers",
                name=elements[i],
            ),
        )
        
    fig.update_layout(
        xaxis=dict(
            title="Velocity (km/s)",
            tickformat = '0d'
        ),
        yaxis=dict(title="Fractional Abundance", 
                   exponentformat="e",
                   range=[0, 1],
#                    side="left"
                  ),
        yaxis2=dict(title="Density", 
                    exponentformat="e",                    
                    overlaying="y",
                    side="right"),
        height=500,
        title=title,
        hovermode="closest",
        legend=dict(
            x=1.15,
        )
    )
    
    display(tbtns_scale, fig)

In [14]:
tbtns_scale = ipw.ToggleButtons(
    options=['Linear', 'Log'],
    description='Scale of yaxes: ',
    style={'description_width': 'initial'},
    value='Linear'
)
# tbtns_scale

In [15]:
def scale_handler(obj):
    if obj.new == 'Linear':
        fig.update_layout(
            yaxis=dict(
                       type="linear",
                       range=[0, 1],
                      ),
            yaxis2=dict(type="linear")
        )
    else:
        fig.update_layout(
            yaxis=dict(
                       type="log",
                       range=[-8, 0],
                      ),
            yaxis2=dict(type="log")
        )

tbtns_scale.observe(scale_handler, names='value')

In [16]:
locked_list = np.array([False] * no_of_elements)

def lock_checkbox_handler(obj):
    abundance_index = obj.owner.name
    shell_no = dropdown.value
    
    if obj.new == True:
        locked_list[abundance_index] = True
        
        # Ensure the sum of locked elements less than 1
        locked_sum = abundance.loc[locked_list, shell_no-1].sum()
        if locked_sum > 1:
            items[abundance_index].value = 1 - (abundance.loc[locked_list, shell_no-1].sum() - items[abundance_index].value)
    
    else:
        locked_list[abundance_index] = False

In [17]:
def update_abundance(obj):
    shell_no = dropdown.value
    element_no = obj.owner.name # start from 0

    # Ensure the sum of locked elements less than 1
    if locked_list[element_no] == True: # if the element is locked
        locked_sum = abundance.loc[locked_list, shell_no-1].sum() - obj.old + obj.new
        if locked_sum > 1:
#             obj.new = 1 - (abundance.loc[locked_list, shell_no-1].sum() - obj.old)
#             obj.owner.value = obj.new
            value = 1 - (abundance.loc[locked_list, shell_no-1].sum() - obj.old)
            obj.owner.value = float("{:.2e}".format(value))
    
    abundance.iloc[element_no, shell_no-1] = obj.new
    
    if abs(abundance.iloc[:, shell_no-1].sum() - 1) < 1.5e-3:
        norm_warning.layout.visibility = 'hidden'
    else:
        norm_warning.layout.visibility = 'visible'
    
    # Update plot
    fig.data[element_no+1].y = abundance.iloc[element_no]
    
def read_abundance(shell_no):
    for i, item in enumerate(items):
#         item.value = abundance.iloc[i, shell_no-1]
        value = abundance.iloc[i,shell_no-1]
        item.value = float("{:.2e}".format(value))

# def display_abundance_df():
#     abundance_view = abundance.copy()
#     abundance_view.index = elements
#     abundance_view.columns=range(1, no_of_shells+1)
#     abundance_view.columns.name = 'Shell No.'
#     display(abundance_view)

In [18]:
# out = ipw.Output()

# Change the shell
def dropdown_eventhandler(obj):
    locked_list = np.array([False] * no_of_elements)
    for lock in locks:
        lock.value = False
    shell_no_new = obj.new
    read_abundance(shell_no_new)
    read_density(shell_no_new)
    
    # Control 'previous' and 'next' buttons.
    if obj.new == 1:
        btn_previous.disabled = True
    else:
        btn_previous.disabled = False
    
    if obj.new == no_of_shells:
        btn_next.disabled = True
    else:
        btn_next.disabled = False

dropdown.observe(dropdown_eventhandler, names='value')

for i, item in enumerate(items):
    item.observe(update_abundance, names='value')
    item.name = i
    locks[i].observe(lock_checkbox_handler, names='value')
    locks[i].name = i

In [19]:
def to_previous_shell(obj):
    dropdown.value -= 1
    
def to_next_shell(obj):
    dropdown.value += 1

def normalize(obj):
    shell_no = dropdown.value
    locked_sum = abundance.loc[locked_list, shell_no-1].sum()
    
    # if abundances are all zero
    if abundance.loc[~locked_list, shell_no-1].sum() == 0:
        return
    
    abundance.loc[~locked_list, shell_no-1] = (1 - locked_sum) * abundance.loc[~locked_list, shell_no-1] / abundance.loc[~locked_list, shell_no-1].sum()
    #abundance.iloc[:, shell_no-1] = abundance.iloc[:, shell_no-1] / abundance.iloc[:, shell_no-1].sum()
    
    read_abundance(shell_no)
    

btn_normalize.on_click(normalize)
btn_previous.on_click(to_previous_shell)
btn_next.on_click(to_next_shell)

In [20]:
# def reset_widgets():
#     pass

In [21]:
help_note = ipw.HTML(value="<p style=\"text-indent: 40px\"><b>Click the checkbox</b> to </p> <p style=\"text-indent: 40px\"> 1) lock the abundance you don't want to normalize </p> <p style=\"text-indent: 40px\"> 2) apply the abundance to other shells.</p>",
                        indent=True
                        )

In [22]:
# Warning message for normalization
norm_warning = ipw.Valid(
    value=False,
    readout='Unnormalized',
    style={'description_width': 'initial'},
    layout=ipw.Layout(visibility='hidden')
)

In [23]:
norm_box = ipw.HBox([btn_normalize, norm_warning])

### Add new elements

In [24]:
import asyncio

class Timer:
    """
    Cited from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
    """
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ 
    Decorator that will postpone a function's
    execution until after `wait` seconds
    have elapsed since the last time it was invoked. 
    Cired from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
    """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [25]:
from pyne import nucname

In [26]:
invalid = ipw.Valid(value=False,
                    layout=ipw.Layout(visibility='hidden')
                    )
symbol_text = ipw.Text(description='Element: ',
                       style={'description_width': 'initial'},
                       placeholder='symbol',
                       layout=ipw.Layout(width='120px'),
                      )

btn_add_element = ipw.Button(icon='plus-square', 
                             description='Add',
                             disabled=True,
                             layout=ipw.Layout(
                                 width='60px'
                             )
                             )

In [27]:
@debounce(0.5)
def symbol_validation(obj):
    element_symbol_string = obj.new.capitalize()
    
    if element_symbol_string in elements:        
        invalid.readout = 'Already exists!'
        invalid.layout.visibility = 'visible'
        btn_add_element.disabled = True
        return
    
    if element_symbol_string == "":
        invalid.layout.visibility = 'hidden' 
        btn_add_element.disabled = True
        return
    
    try:
        if nucname.iselement(element_symbol_string) or nucname.isnuclide(element_symbol_string):
            invalid.layout.visibility = 'hidden'
            btn_add_element.disabled = False
            return

    except RuntimeError:
        pass
    
    invalid.readout = 'invalid'
    invalid.layout.visibility = 'visible'
    btn_add_element.disabled = True
    
symbol_text.observe(symbol_validation, names='value')

In [28]:
def add_element(obj):
    global no_of_elements, no_of_shells, locked_list, elements, abundance_edit_container, main_box
    
    element_symbol_string = symbol_text.value.capitalize()
    
    if element_symbol_string in nucname.name_zz:
        z = nucname.name_zz[element_symbol_string]
        abundance.loc[(z, ''), :] = 0
    else:
        mass_no = nucname.anum(element_symbol_string)
        z = nucname.znum(element_symbol_string)
        abundance.loc[(z, mass_no), :] = 0
    
    abundance.sort_index(inplace=True)
    no_of_elements += 1
    no_of_shells += 1
    
    # Add new BoundedFloatText control and Checkbox control.
    new_item = ipw.BoundedFloatText(min=0, max=1, step=0.01)
    new_lock = ipw.Checkbox(indent=False, layout=ipw.Layout(width='30px'))
    new_item.name = no_of_elements - 1
    new_lock.name = no_of_elements - 1
    items.append(new_item)
    locks.append(new_lock)
    
    locked_list = np.append(locked_list, False)
    
    # Keep the order of description same with atomic number
    elements = get_symbols(abundance)
    for i in range(no_of_elements):
        items[i].description = elements[i]

    abundance_edit_container.children = [ipw.VBox(items), ipw.VBox(locks)]
    
    # Add new trace to plot.
    fig.add_scatter(x=velocity[1:], # convert to km/s
                    y=[0]*no_of_shells,
                    mode="lines+markers",
                    name=element_symbol_string,
                   )
    # Sort the legend in atomic order.
    fig_data_lst = list(fig.data)
    fig_data_lst.insert(np.argwhere(elements == element_symbol_string)[0][0]+1, fig.data[-1])
    fig.data = fig_data_lst[:-1]
    
    read_abundance(dropdown.value)
        
    new_item.observe(update_abundance, names='value')
    new_lock.observe(lock_checkbox_handler, names='value')
    
    # Recover the text boxes.
    symbol_text.value = ''
    
btn_add_element.on_click(add_element)

### Edit abundances in multiple shells

In [29]:
def edit_multiple_shells(btn):
    start_index = shell_range_slider.value[0] - 1
    end_index = shell_range_slider.value[1]
    applied_index = dropdown.value - 1
    abundance_np = abundance.values
    abundance_np[locked_list, start_index:end_index] = abundance_np[locked_list, applied_index].reshape(-1, 1)
    
    # update plot
    update_list = np.where(locked_list == True)[0]
    for i in update_list:
        fig.data[i+1].y = abundance.iloc[i]


In [30]:
btn_multiple_edit = ipw.Button(
    description=' Apply', 
    icon='pencil-square-o',
    layout = ipw.Layout(width='100px')
)

btn_multiple_edit.on_click(edit_multiple_shells)

lbl_apply = ipw.Label(
    description='choosed abundance(s) to next',
    style={'description_width': 'initial'},
#     layout=ipw.Layout(width='230px')
)

shell_range_slider = ipw.IntRangeSlider(
    value=[1, no_of_shells],
    min=1,
    max=no_of_shells,
    step=1,
    description='Shell: ',
    style={'description_width': 'initial'},
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
)

apply_box = ipw.VBox([ipw.HBox([btn_multiple_edit, ipw.Label(value='choosed abundance(s) to')]), shell_range_slider],
                     layout=ipw.Layout(margin='5px 0 0 50px'))

### Edit density profile

In [31]:
ditem = ipw.FloatText(
    description='Density',
    layout=ipw.Layout(width='230px',)
)

dtype_dropdown = ipw.Dropdown(
    options=['-', 'uniform', 'exponential', 'power_law'],
    description='Density type: ',
    style={'description_width': 'initial'},
    layout = ipw.Layout(width='300px')
)

txt_rho_0 = ipw.FloatText(
    description='rho_0',
    layout = ipw.Layout(width='300px')
)

txt_exp = ipw.FloatText(
    description='exponent',
    layout = ipw.Layout(width='300px')
)

txt_v_0 = ipw.FloatText(
    description='v_0',
    layout = ipw.Layout(width='300px')
)

txt_value = ipw.FloatText(
    description='value',
    layout = ipw.Layout(width='300px')
)

btn_dcalculate = ipw.Button(
    icon='calculator',
    description='Calculate Density',
    layout = ipw.Layout(width='300px')
)

uniform_box = ipw.HBox(
    [txt_value, ipw.Label(value='g cm^3')]
)

exp_box = ipw.VBox(
    [ipw.HBox([txt_rho_0, ipw.Label(value='g cm^3')]),
    ipw.HBox([txt_v_0, ipw.Label(value='km/s')])]
)
pow_box = ipw.VBox(
    [ipw.HBox([txt_rho_0, ipw.Label(value='g cm^3')]),
    ipw.HBox([txt_v_0, ipw.Label(value='km/s')]), txt_exp]
)

In [32]:
def read_density(shell_no):
    dvalue = density[shell_no].value
    ditem.value = float("{:.3e}".format(dvalue))

In [33]:
def update_density(obj):
    shell_no = dropdown.value
    density[shell_no] = obj.new * density.unit
    
    # update plot
    fig.data[0].y = density[1:]

ditem.observe(update_density, names='value')

In [34]:
dbox_out = ipw.Output()

@dbox_out.capture(clear_output=True)
def dtype_dropdown_handler(obj):
    if obj.new == 'uniform':
        display(uniform_box)
    elif obj.new == 'exponential':
        display(exp_box)
    elif obj.new == 'power_law':
        display(pow_box)

dtype_dropdown.observe(dtype_dropdown_handler, names='value')

In [35]:
def customize_density(btn):
    dtype = dtype_dropdown.value
    global density
    
    if dtype == '-':
        return
    
    if dtype == 'uniform':
        if txt_value.value == 0:
            return
        
        density = txt_value.value * density.unit * np.ones(no_of_shells)
    else:
        if txt_v_0.value == 0 or txt_rho_0.value == 0:
            return
        
        adjusted_velocity = velocity.insert(0, 0)
        v_middle = (adjusted_velocity[1:] * 0.5 +
                    adjusted_velocity[:-1] * 0.5)
        v_0 = txt_v_0.value * velocity.unit
        rho_0 = txt_rho_0.value * density.unit

        if dtype == 'exponential':
            density = calculate_exponential_density(v_middle, v_0,
                                                          rho_0)

        elif dtype == 'power_law':
            exponent = txt_exp.value
            density = calculate_power_law_density(v_middle, v_0,
                                                        rho_0,
                                                        exponent)
    
    read_density(dropdown.value)

btn_dcalculate.on_click(customize_density)

In [36]:
ditem_box = ipw.HBox(
    [ditem, ipw.Label(value='$\mathrm{g\cdot cm^3}$')], 
    layout=ipw.Layout(margin='0 0 20px 0')
)

### Add new shell

In [37]:
btn_add_shell = ipw.Button(
    icon='plus-square', 
    description=' Insert New Shell',
    layout=ipw.Layout(margin='0 0 0 10px')
)

In [38]:
def add_shell(obj):
    global density, velocity, no_of_shells
    
    if dropdown.value == no_of_shells:
        abundance[no_of_shells] = 0
    else:
        abundance.insert(dropdown.value, '', 0)
        abundance.columns = range(no_of_shells+1)
    no_of_shells += 1
    
    
    # Calculate new velocity
    velocity = quantity_linspace(velocity[0], velocity[-1], len(velocity) + 1).cgs.to('km/s')
    
    # Insert new density
    density = np.insert(density.value, dropdown.value+1, values=0) * density.unit
    
    # Update data and x axis in plot
    with fig.batch_update():
        fig.data[0].x = velocity[1:]
        fig.data[0].y = density[1:]
        for i in range(no_of_elements):
            fig.data[i+1].x = velocity[1:]
            fig.data[i+1].y = abundance.iloc[i]
    
    dropdown.options=list(range(1, no_of_shells+1))

btn_add_shell.on_click(add_shell)

### Display

In [39]:
head = ipw.HBox([dropdown, btn_previous, btn_next, btn_add_shell])
add_element_box = ipw.VBox([ipw.HBox([symbol_text, btn_add_element]), invalid], layout=ipw.Layout(margin='0 0 0 100px'))

# display final output
out = ipw.Output()

In [40]:
density_type_box = ipw.VBox([dtype_dropdown, dbox_out, btn_dcalculate])
feature_box = ipw.VBox([help_note, norm_box, apply_box])
abundance_box = ipw.VBox([add_element_box, ipw.HBox([abundance_edit_container, feature_box])])
density_box = ipw.VBox([ditem_box, density_type_box])
main_tab = ipw.Tab([abundance_box, density_box])
main_tab.set_title(0, 'Edit Abundance')
main_tab.set_title(1, 'Edit Density')

In [41]:
# Display the widget.
with out:
    out.clear_output()
    generate_abundance_density_plot()
    display(plot_output)
    display(head)
    read_density(shell_no=dropdown.value)
    read_abundance(shell_no=dropdown.value)
    display(main_tab)
display(out)

Output()